<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# autoreload updated and newly installed packages
# without having to restart Jupyter kernel
%load_ext autoreload
%autoreload 2
# Some little interaction with matplotlib
%matplotlib inline
# Avoid using Jedi for faster autocomplete (tab)
%config Completer.use_jedi = False

## Crack growth analysis

### Hollow cylinder with crack on the external surface 

In [2]:
# Standard imports
import ast
import datetime
import platform
import os
import sys
from typing import List
# Non-standard imports
from pathlib import Path
# from scipy import stats
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import numba as nb
import numpy as np
import pandas as pd
import plotly.graph_objs as go
# py-fatigue import
import py_fatigue as pf
import py_fatigue.testing as test
# Paths of this tutorial and of the data
TUTORIAL_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(Path.home(), "Documents", "Work", "data")
if not TUTORIAL_PATH in sys.path:
    sys.path.append(TUTORIAL_PATH)
np.set_printoptions(threshold=sys.maxsize)

In [3]:
print(f"Platform: {platform.platform()}")
print(f"Python version: {sys.version}")
print(f"py-fatigue version: {pf.styling.TermColors.CBOLD}{pf.__version__}")

print(f"{pf.styling.TermColors.CWHITEBG}\
{pf.styling.TermColors.CBOLD}\
{pf.styling.TermColors.CBLUE}")
      
print(f"DATA_PATH = {DATA_PATH}")
print(f"TUTORIAL_PATH = {TUTORIAL_PATH}")

print(f"{pf.styling.TermColors.CEND}")

Platform: Linux-5.19.0-26-generic-x86_64-with-glibc2.35
Python version: 3.8.15 (default, Dec 14 2022, 14:18:12) 
[GCC 12.2.0]
py-fatigue version: 1.0.7

DATA_PATH = /home/pd/Documents/Work/data
TUTORIAL_PATH = /home/pd/Python/Packages/Github/py-fatigue



In [4]:
plt.rcParams["figure.figsize"] = (7, 3.5)
plt.rcParams["font.family"] = ["Sans-Serif"]
plt.rcParams["font.size"] = 10
plt.rcParams["lines.markersize"] = 3
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "#DDDDDD"
plt.rcParams["grid.linestyle"] = "-"
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.bottom"] = False
plt.rcParams["xtick.labelbottom"] = True
plt.rcParams["ytick.left"] = False
plt.rcParams["ytick.labelleft"] = True

In [5]:
# 1
print(f"\33[36m1. Read \33[1m.csv\33[22m file\33[0m")
df_ = pd.read_csv(os.path.join(DATA_PATH, "CC_BB_C01_SS_2018_TO_2021.csv"))
# 2
print(f"\33[36m2. Set \33[1m'timestamp'\33[22m as index\33[0m")
df_ = df_.set_index("timestamp")
df_.index = pd.to_datetime(df_.index)
# 3
for col in df_.columns:
    if col.startswith("CC_"):
        tqdm.pandas(desc=f"\33[36m3. Converting \33[1m{col[:3]}{col[-3:]}\33[0m") 
        df_[col] = df_[col].progress_apply(
            lambda x: ast.literal_eval(x) if not x!=x else x
        )
# 4
tqdm.pandas(desc=f"\33[36m4. From \33[1mdict \33[22mto \33[1mdwf.CycleCount\33[0m")
for col in df_.columns:
    if col.startswith("CC_"):
        df_[col] = df_[col].progress_apply(
            lambda x: pf.CycleCount.from_rainflow(x) if not x!=x else x
        )

1. Read .csv file
2. Set 'timestamp' as index


3. Converting CC_nr1: 100%|███████████| 157861/157861 [00:33<00:00, 4769.75it/s]


In [6]:
# 4
tqdm.pandas(desc=f"\33[36m4. From \33[1mdict \33[22mto \33[1mdwf.CycleCount\33[0m")
for col in df_.columns:
    if col.startswith("CC_"):
        df_[col] = df_[col].progress_apply(
            lambda x: pf.CycleCount.from_rainflow(x) if not x!=x else x
        )

4. From dict to dwf.CycleCount: 100%|█| 157861/157861 [00:31<00:00, 5013.43it/s]


In [ ]:
x